In [1]:
import lda
import numpy as np

## Algorithm

使用LDA进行文本生成的过程：

写M篇文章，一共涉及了K个主题，每一个topic下的词部分为一个$\beta$的Dirichlet先验分布中sample出来的Multinomial分布。

对于每一篇文章，会从一个泊松分布中sample一个长度作为文章的长度，再从一个参数为$\alpha$的Dirichlet先验分布中sample出一个Multinomial分布作为该文章在每一个topic下词的概率。

当想写某一篇文章中的第n个词的时候，首先从该文章中出现每一个Topic的Multinomial分布中sample一个Topic，然后再在这个Topic对应当词的Multinomial分布中sample一个词，作为要填写的词。

```
# Topic plate
for all topics $k \in [1,K]$:
    sample mixture components $\phi_k \sim Dir(\beta)$
# Document plate
for all documents $m \in [1,M]$:
    sample mixture proportion $\theta_m \sim Dir(\alpha)$
    sample document length $N_m \sim Poiss(\zeta)$
    # Word plate
    for all words in $n \in [1,N_m]$ in document m do:
        sample topic index $z_{m,n} \sim Mult(\theta_m)$
        sample term for word $w_{m,n} \sim Multi(\phi_{m,n})$
```

In [2]:
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()

In [3]:
print X.shape

(395, 4258)


In [4]:
X

array([[1, 0, 1, ..., 0, 0, 0],
       [7, 0, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int32)

In [75]:
print vocab

('church', 'pope', 'years', 'people', 'mother', 'last', 'told', 'first', 'world', 'year', 'president', 'teresa', 'charles', 'catholic', 'during', 'life', 'u.s', 'city', 'public', 'time', 'since', 'family', 'king', 'former', 'british', 'harriman', 'against', 'country', 'vatican', 'made', 'three', 'hospital', 'minister', 'home', 'died', 'tuesday', 'government', "n't", 'million', 'prince', 'john', 'very', 'war', 'say', 'order', 'day', 'political', 'leader', 'heart', 'roman', 'yeltsin', 'later', 'france', 'party', 'monday', 'death', 'clinton', 'sunday', 'elvis', 'state', 'diana', 'royal', 'wednesday', 'churchill', 'official', 'service', 'including', 'film', 'while', 'week', 'left', 'work', 'paris', 'around', 'american', 'saying', 'take', 'doctors', 'group', 'son', 'queen', 'house', 'police', 'national', 'under', 'surgery', 'next', 'peace', 'paul', 'long', 'ceremony', 'month', 'thursday', 'french', 'head', 'marriage', 'operation', 'visit', 'show', 'germany', 'russian', 'days', 'love', 'news

In [6]:
print titles

('0 UK: Prince Charles spearheads British royal revolution. LONDON 1996-08-20', '1 GERMANY: Historic Dresden church rising from WW2 ashes. DRESDEN, Germany 1996-08-21', "2 INDIA: Mother Teresa's condition said still unstable. CALCUTTA 1996-08-23", '3 UK: Palace warns British weekly over Charles pictures. LONDON 1996-08-25', '4 INDIA: Mother Teresa, slightly stronger, blesses nuns. CALCUTTA 1996-08-25', "5 INDIA: Mother Teresa's condition unchanged, thousands pray. CALCUTTA 1996-08-25", '6 INDIA: Mother Teresa shows signs of strength, blesses nuns. CALCUTTA 1996-08-26', "7 INDIA: Mother Teresa's condition improves, many pray. CALCUTTA, India 1996-08-25", '8 INDIA: Mother Teresa improves, nuns pray for "miracle". CALCUTTA 1996-08-26', '9 UK: Charles under fire over prospect of Queen Camilla. LONDON 1996-08-26', '10 UK: Britain tells Charles to forget Camilla. LONDON 1996-08-27', "11 COTE D'IVOIRE: FEATURE - Quiet homecoming for reprieved Ivory Coast maid. ABIDJAN 1996-08-28", '12 INDIA: 

In [97]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."

doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

In [8]:
import nltk
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [9]:
raw = doc_a.lower()
tokens = tokenizer.tokenize(raw)
print tokens

['brocolli', 'is', 'good', 'to', 'eat', 'my', 'brother', 'likes', 'to', 'eat', 'good', 'brocolli', 'but', 'not', 'my', 'mother']


In [10]:
from stop_words import get_stop_words
en_stop = get_stop_words('en')

In [11]:
stopped_tokens = [i for i in tokens if not i in en_stop]
print stopped_tokens

['brocolli', 'good', 'eat', 'brother', 'likes', 'eat', 'good', 'brocolli', 'mother']


In [12]:
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [16]:
# stem token
texts = [p_stemmer.stem(i) for i in stopped_tokens]
print texts

['brocolli', 'good', 'eat', 'brother', u'like', 'eat', 'good', 'brocolli', 'mother']


In [18]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

In [20]:
print dictionary

Dictionary(32 unique tokens: [u'often', u'feel', u'profession', u'drive', u'say']...)


In [21]:
print corpus

[[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2)], [(3, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(8, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(3, 1), (4, 1), (8, 1), (18, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)], [(0, 1), (1, 1), (19, 2), (30, 1), (31, 1)]]


In [22]:
print ldamodel

LdaModel(num_terms=32, num_topics=2, decay=0.5, chunksize=2000)


In [85]:
import numpy as np
from nltk.tokenize import RegexpTokenizer

doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]


V = 10
K = 10
M = 10
N = 10
z = np.zeros((M, N))
doc = np.zeros((M,N))

nmk = np.zeros((M,K))
nkt = np.zeros((K,V))
nmkSum = np.zeros(M)
nktSum = np.zeros(K)
phi = np.zeros((K,V))
theta = np.zeros((M,K))

iteration = 100
saveStep = 20
beginSaveIters = 5

alpha = 3
beta = 3

In [77]:
print doc_set

['Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother.', 'My mother spends a lot of time driving my brother around to baseball practice.', 'Some health experts suggest that driving may cause increased tension and blood pressure.', 'I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better.', 'Health professionals say that brocolli is good for your health.']


In [83]:
def initial():
    global nmk
    global nkt
    global nmkSum
    global nktSum
    global phi
    global theta
    global z
    global doc
    
    tokenizer = RegexpTokenizer(r'\w+')
    dict_ = {}
#     for m in range(M):
        
    for m in range(M):
        for n in range(N):
            initTopic = int(random.random()*K)
            z[m, n] = initTopic
            nmk[m, initTopic] += 1
            nkt[initTopic, doc[m, n]] += 1
            nktSum[initTopic] += 1
        nmkSum[m] = N
    return

initial()
print z


[[ 3.  7.  7.  9.  9.  5.  2.  7.  7.  3.]
 [ 8.  5.  7.  5.  4.  4.  5.  3.  2.  5.]
 [ 0.  1.  1.  2.  1.  0.  7.  8.  2.  2.]
 [ 1.  4.  0.  2.  7.  3.  0.  0.  2.  8.]
 [ 9.  2.  8.  6.  7.  6.  7.  2.  1.  2.]
 [ 6.  4.  5.  8.  3.  5.  2.  2.  4.  3.]
 [ 1.  6.  8.  1.  8.  6.  7.  2.  3.  1.]
 [ 4.  7.  1.  1.  9.  0.  5.  1.  1.  6.]
 [ 5.  5.  1.  9.  6.  4.  5.  5.  9.  6.]
 [ 8.  0.  8.  8.  4.  8.  3.  1.  0.  9.]]


In [94]:
def inference():
    for i in range(iteration):
        if (i-beginSaveIters) % saveStep == 0:
            updateEstimatedParameters()
        for m in range(M):
            for n in range(N):
                newTopic = sampleTopicZ(m,n)
                z[m,n] = newTopic
    return

def sampleTopicZ(m,n):
    # sample from p(z_i|z_-1, w) using Gibbs update rule
    oldTopic = z[m,n]
    nmk[m,oldTopic] -= 1
    nkt[oldTopic, doc[m][n]] -= 1
    nmkSum[m] -= 1
    nktSum[oldTopic] -= 1
    
    # compute p(z_i ] k | z_-1, w)
    p = np.zeros(K)
    for k in range(K):
        p[k] = ( nkt[k, doc[m,n]] + beta ) / ( nktSum[k] + V*beta ) * (nmk[m,k] + alpha) / (nmkSum[m] + k*alpha)
        
    # sample a new topic label for w_{m,n}
    for k in range(K):
        p[k] += p[k-1]
    u = random.random() * p[k-1]
    for newTopic in range(K):
        if u < p[newTopic]:
            break
            
    nmk[m, newTopic] += 1
    nkt[newTopic, doc[m,n]] += 1
    nmkSum[m] += 1
    nktSum[newTopic] += 1
    return newTopic

def updateEstimatedParameters():
    global phi
    global theta
    
    for k in range(K):
        for t in range(V):
            phi[k,t] = (nkt[k,t] + beta) / (nktSum[k] + V*beta)
            
    for m in range(M):
        for k in range(K):
            theta[m,k] = (nmk[m,k] + alpha) / (nmkSum[m] + K*alpha)
    
    return

In [96]:
print theta

initial()
inference()

print
print theta

[[ 1.425  0.1    0.075  0.1    0.125  0.075  0.075  0.075  0.125  0.075]
 [ 1.35   0.2    0.075  0.1    0.075  0.075  0.075  0.075  0.15   0.075]
 [ 1.4    0.225  0.1    0.075  0.075  0.075  0.075  0.075  0.075  0.075]
 [ 1.025  0.45   0.125  0.075  0.075  0.125  0.075  0.075  0.15   0.075]
 [ 1.35   0.2    0.15   0.075  0.075  0.1    0.075  0.075  0.075  0.075]
 [ 1.     0.375  0.2    0.1    0.075  0.075  0.125  0.075  0.15   0.075]
 [ 1.175  0.25   0.225  0.1    0.1    0.075  0.075  0.075  0.1    0.075]
 [ 1.05   0.475  0.175  0.075  0.075  0.075  0.075  0.075  0.1    0.075]
 [ 1.4    0.15   0.125  0.075  0.075  0.075  0.075  0.075  0.125  0.075]
 [ 1.325  0.125  0.175  0.1    0.075  0.075  0.075  0.075  0.15   0.075]]

[[ 1.65   0.1    0.125  0.1    0.1    0.075  0.075  0.075  0.125  0.075]
 [ 1.55   0.225  0.1    0.1    0.075  0.075  0.075  0.075  0.15   0.075]
 [ 1.525  0.325  0.1    0.075  0.075  0.075  0.1    0.075  0.075  0.075]
 [ 1.1    0.625  0.175  0.075  0.075  0.1    0.07